In [1]:
#import libraies
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

In [2]:
import warnings
warnings.filterwarnings("ignore") # gereksiz uyarıları kaldıralım 

In [3]:
# squad veri seti uzerinde ince ayar yapilmis bert modeli
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

In [4]:
# bert tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

In [5]:
# soru cevaplama gorevi icin ince ayar yapilmis bert modeli
model = BertForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# Soru-Cevap Tahmini Fonksiyonu

def predict_answer(context, question):
    """
    verilen bir metin (context) ve bir soru (question) kullanarak
    metin içerisinden cevabı tahmin eder.

    adım adım ne yaptığı şu şekilde özetlenebilir:

    1. metin ve soruyu tokenize eder.
    2. token'lar üzerinden modele uygun input verilerini oluşturur.
    3. modeli çalıştırarak, cevabın başlayabileceği ve bitebileceği yerlerin skorlarını alır.
    4. bu skorlar arasından en yüksek skora sahip başlangıç ve bitiş indekslerini seçer.
    5. ilgili token ID'lerini metin haline getirerek cevabı çıkarır.
    6. cevabı okunabilir bir string formatına dönüştürerek döner.
    """

    # 1. encode_plus ile hem soru hem metin birlikte token'lara ayrılır.
    # max_length: 512, çünkü transformer'lar genelde maksimum bu kadar token alabilir.
    encoding = tokenizer.encode_plus(
        question,
        context,
        return_tensors="pt",
        max_length=512,
        truncation=True
    )

    # 2. giriş verileri hazırlanır
    input_ids = encoding["input_ids"]  # token ID'leri
    attention_mask = encoding["attention_mask"]  # dikkate alınacak token'lar maskelenir

    # 3. model çalıştırılır, ama gradyan hesabı yapmadan (inference modu)
    with torch.no_grad():
        start_scores, end_scores = model(
            input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )

    # 4. skorlar içerisinden en yüksek olasılıkların indeksleri alınır
    start_index = torch.argmax(start_scores, dim=1).item()  # cevabın başlayabileceği en yüksek skorlu index
    end_index = torch.argmax(end_scores, dim=1).item()  # cevabın bitebileceği en yüksek skorlu index

    # 5. token ID'lerini alarak cevabın geçtiği token'ları bul
    answer_tokens = tokenizer.convert_ids_to_tokens(
        input_ids[0][start_index:end_index + 1]
    )

    # 6. token'ları okunabilir string haline getiriyoruz
    answer = tokenizer.convert_tokens_to_string(answer_tokens)

    return answer

In [7]:
question = "What is the capital of France"
context = "France, officially the French Republic, is a country whose capital is Paris"

answer = predict_answer(context, question)
print(f"Answer: {answer}")

Answer: paris


In [8]:

question = '''What is Machine Learning?'''
context = ''' Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance 
                on a specific task. Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or 
                decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection 
                of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning 
                is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, 
                theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory 
                data analysis through unsupervised learning.In its application across business problems, machine learning is also referred to as predictive analytics. '''


answer = predict_answer(context, question)
print(f"Answer: {answer}")


Answer: the scientific study of algorithms and statistical models
